# Week 2 coding tasks
## J.Merritt 2022_09_14

**1. First, import the primary care physicians dataset (`primary_care_physicians.csv`) into a data frame named `physicians`.**

In [1]:
import pandas as pd

In [8]:
physicians = pd.read_csv('../data/primary_care_physicians.csv')

In [3]:
physicians.head()

,FIPS,state,county,primary_care_physicians
0,1001,Alabama,Autauga,26.0
1,1003,Alabama,Baldwin,153.0
2,1005,Alabama,Barbour,8.0
3,1007,Alabama,Bibb,12.0
4,1009,Alabama,Blount,12.0


**2. Filter `physicians` down to just the counties in Tennessee. Save the filtered dataframe back to `physicians`. Verify that the resulting dataframe has 95 rows.**

In [9]:
physicians = physicians.loc[physicians['state'] == 'Tennessee']

In [10]:
physicians.info

<bound method DataFrame.info of        FIPS      state      county  primary_care_physicians
2432  47001  Tennessee    Anderson                     39.0
2433  47003  Tennessee     Bedford                     15.0
2434  47005  Tennessee      Benton                      3.0
2435  47007  Tennessee     Bledsoe                      1.0
2436  47009  Tennessee      Blount                     90.0
...     ...        ...         ...                      ...
2522  47181  Tennessee       Wayne                      5.0
2523  47183  Tennessee     Weakley                     18.0
2524  47185  Tennessee       White                      9.0
2525  47187  Tennessee  Williamson                    338.0
2526  47189  Tennessee      Wilson                     43.0

[95 rows x 4 columns]>

**3. Look at the distribution of the number of primary care physicians. What do you notice?**
#### There is a massive variation in the number of pcp's between counties

In [11]:
physicians.describe()

,FIPS,primary_care_physicians
count,95.000000,95.000000
mean,47095.000000,51.042105
std,55.136195,129.311426
min,47001.000000,0.000000
25%,47048.000000,4.500000
50%,47095.000000,12.000000
75%,47142.000000,26.500000
max,47189.000000,806.000000


**4. Now, import the population by county dataset (`population_by_county.csv`) into a DataFrame named `population`.**

In [38]:
population = pd.read_csv('../data/population_by_county.csv')

In [13]:
population.head()

,FIPS,population,county,state,urban
0,17051,21565,Fayette County,ILLINOIS,Rural
1,17107,29003,Logan County,ILLINOIS,Rural
2,17165,23994,Saline County,ILLINOIS,Rural
3,17097,701473,Lake County,ILLINOIS,Urban
4,17127,14219,Massac County,ILLINOIS,Rural


**5. Merge the `physicians` DataFrame with the `population` DataFrame. Keep only the values for Tennessee. When you merge, be sure the include both the `population` and `urban` columns in the merged results. Save the result of the merge back to `physicians`.**

In [43]:
physicians = pd.merge(left = physicians, 
                      right = population[['FIPS', 'population', 'urban']], 
                      on = 'FIPS')

In [36]:
# alternative:
# population['county'] = population['county'].str.replace(' County', '')
# physicians = pd.merge(left = physicians, 
#                 right = population.loc[population['state'] 
#                                        == 'TENNESSEE'][['population', 'urban', 'county']], 
#                 on = 'county'
#                 )

**6. How many Tennessee counties are considered urban?**
#### 38

In [49]:
len(physicians.loc[physicians['urban'] == 'Urban'])

38

**7. The State Health Access Data Assistance Center (SHADAC) (https://www.shadac.org/) classifies counties into three groups based on the number of residents per primary care physician. First, counties with fewer than 1500 residents per primary care physician are considered to have an "adequate" supply. Counties with at least 1500 residents but fewer than 3500 residents per primary care physician are considered to have a "moderately inadequate" supply, and counties with at least 3500 residents per primary care physician are considered to have a "low inadequate" supply. How many counties in Tennessee are in each group?** 

In [77]:
physicians['residents_per_pcp'] = physicians['population']/physicians['primary_care_physicians']

In [115]:
tn_health_access = pd.DataFrame()

In [109]:
adequate = len(physicians.loc[physicians['residents_per_pcp'] < 1500])

moderately_inadequate = len(physicians.loc[(physicians['residents_per_pcp'] >= 1500) & 
                        (physicians['residents_per_pcp'] < 3500)])
    
low_inadequate = len(physicians.loc[(physicians['residents_per_pcp'] >= 3500) | 
                        (physicians['primary_care_physicians'] == 0)])

In [116]:
tn_health_access=pd.DataFrame({"health_access":['adequate','moderately_inadequate','low_inadequate'],
                 "number_counties":[adequate, moderately_inadequate, low_inadequate]})

In [117]:
tn_health_access

,health_access,number_counties
0,adequate,14
1,moderately_inadequate,50
2,low_inadequate,31


**alternative approach**

In [119]:
physicians.loc[physicians['residents_per_pcp'] < 1500].describe() 

,FIPS,primary_care_physicians,population,residents_per_pcp
count,14.000000,14.000000,14.000000,14.000000
mean,47093.857143,253.571429,245683.500000,1107.774979
std,62.588838,257.329395,274574.154396,346.822923
min,47009.000000,22.000000,31732.000000,565.508850
25%,47035.500000,43.000000,63773.750000,813.581600
50%,47086.000000,164.000000,128866.000000,1097.785442
75%,47153.000000,386.750000,327036.500000,1461.499242
max,47187.000000,806.000000,936374.000000,1492.135135


In [121]:
physicians.loc[(physicians['residents_per_pcp'] >= 1500) & 
               (physicians['residents_per_pcp'] < 3500)].describe() 

,FIPS,primary_care_physicians,population,residents_per_pcp
count,50.000000,50.000000,50.000000,50.000000
mean,47098.120000,23.680000,53141.680000,2364.119813
std,57.110219,24.486098,54748.067992,467.391836
min,47001.000000,2.000000,5079.000000,1545.370370
25%,47047.500000,9.250000,22979.750000,1957.230769
50%,47106.000000,17.000000,38606.500000,2334.011905
75%,47146.500000,26.000000,53528.000000,2720.706731
max,47189.000000,137.000000,315815.000000,3338.600000


In [125]:
physicians.loc[(physicians['residents_per_pcp'] >= 3500) | 
               (physicians['primary_care_physicians'] == 0)].describe()

,FIPS,primary_care_physicians,population,residents_per_pcp
count,31.000000,31.000000,31.000000,31.000000
mean,47090.483871,3.709677,19764.612903,inf
std,49.625176,3.908909,13767.325639,NaN
min,47005.000000,0.000000,5760.000000,3524.600000
25%,47064.000000,1.500000,10956.500000,4189.750000
50%,47085.000000,3.000000,16140.000000,5841.000000
75%,47125.000000,5.000000,24327.500000,6887.333333
max,47175.000000,17.000000,61447.000000,inf


**8. Does there appear to be any detectable relationship between whether a county is urban or rural and its supply of primary care physicians?**

In [137]:
rural = physicians.loc[physicians['urban'] == 'Rural']['county']
urban = physicians.loc[physicians['urban'] == 'Urban']['county']

In [ ]:
(physicians['county'].isin(rural))
(physicians['county'].isin(urban))

In [141]:
physicians.loc[(physicians['residents_per_pcp'] < 1500) & (physicians['county'].isin(rural))].describe()

,FIPS,primary_care_physicians,population,residents_per_pcp
count,5.000000,5.000000,5.000000,5.000000
mean,47062.200000,34.800000,51177.600000,1461.801448
std,48.961209,12.557866,19403.923039,46.920864
min,47025.000000,22.000000,31732.000000,1379.652174
25%,47031.000000,23.000000,32284.000000,1467.454545
50%,47035.000000,37.000000,55209.000000,1480.400000
75%,47079.000000,40.000000,59216.000000,1489.365385
max,47141.000000,52.000000,77447.000000,1492.135135


In [142]:
physicians.loc[(physicians['residents_per_pcp'] < 1500) & (physicians['county'].isin(urban))].describe()

,FIPS,primary_care_physicians,population,residents_per_pcp
count,9.000000,9.000000,9.000000,9.000000
mean,47111.444444,375.111111,353742.333333,911.093607
std,64.751662,246.970871,292472.943977,269.278617
min,47009.000000,90.000000,97625.000000,565.508850
25%,47065.000000,199.000000,129927.000000,756.782946
50%,47113.000000,338.000000,225389.000000,886.738462
75%,47163.000000,520.000000,461104.000000,1033.816541
max,47187.000000,806.000000,936374.000000,1443.633333


In [143]:
physicians.loc[((physicians['residents_per_pcp'] >= 3500) | 
               (physicians['primary_care_physicians'] == 0)) & (physicians['county'].isin(rural))].describe()

,FIPS,primary_care_physicians,population,residents_per_pcp
count,21.000000,21.000000,21.000000,21.00
mean,47084.523810,2.666667,16589.285714,inf
std,43.094801,2.415229,10385.446510,NaN
min,47005.000000,0.000000,5760.000000,3524.60
25%,47067.000000,1.000000,7962.000000,4082.00
50%,47085.000000,2.000000,14399.000000,6360.75
75%,47117.000000,4.000000,21545.000000,12994.50
max,47175.000000,9.000000,46064.000000,inf


In [144]:
physicians.loc[((physicians['residents_per_pcp'] >= 3500) | 
               (physicians['primary_care_physicians'] == 0)) & (physicians['county'].isin(urban))].describe()

,FIPS,primary_care_physicians,population,residents_per_pcp
count,10.000000,10.000000,10.000000,10.000000
mean,47103.000000,5.900000,26432.800000,5699.389608
std,61.845686,5.486347,17865.683193,2856.444130
min,47015.000000,1.000000,10231.000000,3614.529412
25%,47061.000000,3.000000,14931.000000,4378.175000
50%,47096.000000,4.000000,21294.500000,4844.300000
75%,47165.500000,5.000000,24570.250000,5749.250000
max,47173.000000,17.000000,61447.000000,13427.000000
